In [1]:
import sys, os
sys.path.append('..') # One step back from the currect direction

from utils.prompts import render # Visualized the prompts
from utils.llm_client import LLMClient # pick LLM Client
from utils.logging_utils import log_llm_call # logging every API call
from utils.router import pick_model, should_use_reasoning_model # General, reasoning or strong

In [2]:
os.getcwd()

'c:\\Users\\user\\Downloads\\ZUU CREW\\AI Engineer Essentials\\Codes\\Week 01\\notebooks'

### 01 - Zero Shot

In [3]:
prompt_text, spec = render(
                        "zero_shot.v1",
                        role = "sentiment_analysis",
                        Instruction = "Analyze the following text determine the sentiment as positive / negetive / neutral.",
                        constrains = "The sentiment should be one of the following: positive, negetive, neutral.",
                        format = "The sentiment is: {sentiment}"
                        )

model = pick_model("openai", "general") # Hard to pick general or reasoning but choose general because detecting sentiment is not that logical, it's mainly depends on the keywords.
print("Selected model:", model)
llm = LLMClient("openai", model)

text = "I'm really happy with the product! It's amazing!"
messages = [
            {
                "role" : "user",
                "content" : f"{prompt_text}\n\nReview: {text}"
            }
]

llm.chat(messages, temperature=0.0)

Selected model: gpt-4o-mini


{'text': 'The sentiment is: positive',
 'usage': {'input_tokens_est': 47,
  'context_tokens_est': 0,
  'total_est': 50,
  'prompt_tokens_actual': 50,
  'completion_tokens_actual': 5,
  'total_tokens_actual': 55},
 'latency_ms': 2593,
 'raw': ChatCompletion(id='chatcmpl-CsVTOvUrcrZUyaSswbMLo1qKHBQA4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The sentiment is: positive', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1767106730, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_3683ee3deb', usage=CompletionUsage(completion_tokens=5, prompt_tokens=50, total_tokens=55, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))),
 'meta': {'retry_count': 0,

### 02 - Few Shot

In [12]:
examples = """
Example 01:
Review: I'm really happy with the product! It's amazing!
Sentiment: positive
Explanation: User says product is good also he is happy.

Example 02:
Review: I'm really unhappy with the product! It's bad!
Sentiment: negetive
Explanation: User says product is bad and he is unhappy.

Example 03:
Review: The product is okay, but It's not great.
Sentiment: neutral
Explanation: User says product is okay but not great.

Example 04:
Review: I'm not sure about the product.
Sentiment: neutral
Explanation: User is not sure about the product.

Example 05:
Review: I'm really happy with the product! It's bad!
Sentiment: negetive
Explanation: User says product is good but he is unhappy. So we are prioratizing his words on product quality, not his happiness. That's why we are considering the product quality as negative.

Example 06:
Review: I'm really unhappy with the product but It's amazing!
Sentiment: positive
Explanation: User says product is bad but he is happy. So we are prioratizing his words on product quality, not his happiness. That's why we are considering the product quality as positive.
"""

prompt_text, spec = render(
                        "few_shot.v1",
                        role = "sentiment_analysis", 
                        examples = examples,
                        Instruction = "Analyze the following text determine the sentiment as positive / negetive / neutral. Also provide the explanation for the sentiment.",
                        constrains = "The sentiment should be one of the following: positive, negetive, neutral.",
                        format = "The sentiment is: {sentiment}"
                        )

model = pick_model("openai", "general")
llm = LLMClient("openai", model)

text = "It seems nice but It's not for me."
messages = [
            {
                "role" : "user",
                "content" : f"{prompt_text}\n\nReview: {text}"
            }
]

response = llm.chat(messages, temperature=0.0)
print(response['text'])

The sentiment is: neutral  
Explanation: User expresses that the product seems nice but indicates it is not suitable for them, leading to a neutral sentiment.


### 03 - COT

In [5]:
# import os
# print(os.getenv("GEMINI_API_KEY"))

In [6]:
# model = pick_model("google","reason")
# llm = LLMClient("google", model)

# problem = """ 
#             A car travels 100 miles in 2 hours. What is the average speed of the car?
#             also if car stops for 40 minutes what is the average speed of the car? 
#         """

# prompt_text, spec = render(
#                             "cot_reasoning.v1",
#                             role = "math_tutor",
#                             Instruction = """ Solve the following problem step by step.
#                                                 1. First identify whether car travelled the entire time without stopping or not.
#                                                 2. If car stopped for x minutes and overall for y the travel duration is y-x.
#                                             """,
#                             problem = problem
#                             )

# messages = [
#             {
#                 "role" : "user",
#                 "content" : f"{prompt_text}"
#             }
# ]

# response = llm.chat(messages, temperature=0.3)
# response


### 04 - TOT